In [ ]:
%matplotlib widget

import os
import os.path
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf

data_path = os.path.expandvars('C:/Users/$USERNAME/Desktop/data/07_deep_learning/')

## 1: Data Generator

In [ ]:
from tensorflow.keras.utils import Sequence

class GenData(Sequence):
    def __init__(self, path, split='train', batch_size=32, fraction=5):
        self.split = split
        self.path = os.path.join(path, split, 'output')
        self.batch_size = batch_size
        self.fraction = fraction

        # TODO: lists containing data
        self.x = [] # features: zero crossing, mfccs, ...
        self.y = [] # speech activity: 0 / 1

        # TODO: collect file names under *self.path*
        self.files = ...

        # only select a *fraction* of the file names to limit amount of 
        # training data
        self.files = self.files[1 : int(len(self.files) / self.fraction)]

        for file in tqdm(self.files):
            # TODO: load file & save contents in *self.x* & *self.y*
            pass

        # Random order indices
        self.idx = np.random.permutation(range(len(self.x)))

    def __getitem__(self, item):
        # get random indices of current batch
        idx_batch = self.idx[item*self.batch_size : (item+1)*self.batch_size]

        # TODO: get data for current batch from *self.x* / *self.y*
        x_batch = ...
        y_batch = ...

        # TODO: find minimum length of data for current batch
        minlen = ...

        # TODO: stack data for current batch to tensors of proper size
        x_tensor = ...
        y_tensor = ...

        return x_tensor, y_tensor

    def __len__(self):
        return math.floor(len(self.x) / self.batch_size)

    def on_epoch_end(self):
        if self.split == 'train':
            # reshuffle random order indices
            self.idx = np.random.permutation(range(len(self.x)))

In [ ]:
# Instantiate one generator for training, validation and test data

train_data = GenData(data_path, split='train', batch_size=32)
val_data = GenData(data_path, split='val', batch_size=32)
test_data = GenData(data_path, split='test', batch_size=32)

In [ ]:
# draw one batch from the training generator
x_train_0, y_train_0 = train_data[1]
print(x_train_0.shape)
print(y_train_0.shape)

# check length of training generator
print(len(train_data))

# check if we can iterate whole generator
for x, y in train_data:
    pass

# plot targets for 1st sentence of a batch
plt.plot(y[0,:])

## 2: Feedforward Network

In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import ... # TODO: which loss should be used?

def build_feedforward_model(num_neurons=128, num_blocks=2, dropout_rate=0.15, learning_rate=1E-4):

    model = Sequential()

    # TODO: use ALL input arguments!

    return model

In [ ]:
# test for 1 epochs ...
model_ff = build_feedforward_model()
history_ff = model_ff.fit(train_data, validation_data=val_data, epochs=1)

## 3. Evaluation Function

In [ ]:
def evaluate(model, test_data):
    # TODO

In [ ]:
evaluate(model_ff, test_data)

## 4. First Real Training

In [ ]:
# ... and with 15 more epochs
model_ff = build_feedforward_model()
history_ff = model_ff.fit(train_data, validation_data=val_data, epochs=14)
evaluate(model_ff, test_data)

## 5: A Recurrent Model

In [ ]:
from tensorflow.keras.layers import LSTM

def build_lstm_model(num_neurons=32, num_blocks=2, learning_rate=1E-4):

    model = Sequential()

    # TODO: construct & compile model

    return model

In [ ]:
# test with 1 epoch ...
model_lstm = build_lstm_model()
history_lstm = model_lstm.fit(train_data, validation_data=val_data, epochs=1)
evaluate(model_lstm, test_data)

In [ ]:
# ... and with 15
model_lstm = build_lstm_model()
history_lstm = model_lstm.fit(train_data, validation_data=val_data, epochs=15)
evaluate(model_lstm, test_data)

## 6: Different Learning Rates

In [ ]:
learning_rates = [1E-4, 1E-3, 1E-2]
histories = []

# TODO: build and train models for different learning rates

In [ ]:
# TODO: compare loss over epoch for different learning rates